# 1. Backfill Pipeline

## 1.1. Setup

In [1]:
# Standard imports
import os
import sys
import json
import time
from datetime import date, timedelta
import warnings
from pathlib import Path
warnings.filterwarnings("ignore", module="IPython")

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata

#  Load settings 
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Project root dir: c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!
2026-01-09 10:28:48,243 INFO: Initializing external client
2026-01-09 10:28:48,243 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-09 10:28:49,091 WARNING: UserWarning: The installed hopsworks client version 4.1.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-09 10:28:50,087 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


Repository management

In [2]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

Already in repo at c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn


In [3]:
today = date.today()

if settings.AQICN_API_KEY is None:
    print("AQICN_API_KEY missing.")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
except Exception:
    pass

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

## 1.2. Create Feature Groups

In [4]:
air_quality_fg, sensor_metadata_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 1.3. Check and Backfill
When performed for the first time, might take a long time if many added sensors (about 1.5 minutes per sensor).

Thereafter about 3 seconds per sensor.

In [ ]:
data_dir = os.path.join(root_dir, "data")
dir_list = os.listdir(data_dir)

# Get already processed sensors from feature group
try:
    existing_sensors = set(air_quality_fg.read()["sensor_id"].unique())
    print(f"📋 Found {len(existing_sensors)} sensors already in feature store")
except:
    existing_sensors = set()
    print("📋 No existing sensors found, starting fresh")

# Load all metadata (needed for nearby sensor calculations and location_id lookup)
try:
    metadata_df = sensor_metadata_fg.read()
    if len(metadata_df) > 0:
        # Ensure sensor_id is int type for consistent lookups
        metadata_df["sensor_id"] = metadata_df["sensor_id"].astype(int)
        metadata_df = metadata_df.set_index("sensor_id")
        print(f"📍 Loaded metadata for {len(metadata_df)} sensors")
except:
    metadata_df = pd.DataFrame()
    print("📍 No metadata found - please run pipeline 0 first!")

# Count total sensors to process
total_sensors = len([f for f in dir_list if f.endswith(".csv")])
remaining = total_sensors - len(existing_sensors)
print(f"📊 Total sensors: {total_sensors}, Already processed: {len(existing_sensors)}, Remaining: {remaining}")

# Track processing stats
successful = 0
failed = 0
skipped = 0
failed_sensors = []  # Track which sensors failed and why

for file in dir_list:
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(data_dir, file)
    
    try:
        aq_df_raw, street, city, country, feed_url, sensor_id = metadata.read_sensor_data(
            file_path, AQICN_API_KEY
        )
        
        # Ensure sensor_id is int for consistent comparison
        sensor_id = int(sensor_id)

        # Skip if already processed
        if sensor_id in existing_sensors:
            skipped += 1
            if skipped <= 3:  # Only print first 3 to avoid spam
                print(f"⏩ Skipping sensor {sensor_id}, already in feature store")
            elif skipped == 4:
                print(f"⏩ ... (suppressing further skip messages)")
            continue

        # Clean AQ
        aq_df = cleaning.clean_and_append_data(aq_df_raw, sensor_id)
        aq_df = aq_df.sort_values("date")
        
        # Check for and remove duplicate dates
        initial_rows = len(aq_df)
        aq_df = aq_df.drop_duplicates(subset=["date"], keep="first")
        duplicates_removed = initial_rows - len(aq_df)
        if duplicates_removed > 0:
            print(f"⚠️  Sensor {sensor_id}: Removed {duplicates_removed} duplicate date entries")

        aq_df = feature_engineering.add_lagged_features(aq_df, "pm25", lags=[1,2,3])
        aq_df = feature_engineering.add_rolling_window_feature(aq_df, window_days=3, column="pm25", new_column="pm25_rolling_3d")

        # Only add nearby sensor feature if we have existing metadata
        if len(metadata_df) > 0:
            aq_df = feature_engineering.add_nearby_sensor_feature(aq_df, metadata_df, n_closest=3)
        else:
            # Fill with zeros if no existing sensors to compare against
            aq_df["pm25_nearby_avg"] = 0.0

        # Compute date range
        end_date = aq_df["date"].max().date()
        start_date = end_date - timedelta(days=365 * 3)

        # Get metadata from sensor_metadata_fg (should always exist now)
        if sensor_id in metadata_df.index:
            meta = metadata_df.loc[sensor_id]
            latitude = meta["latitude"]
            longitude = meta["longitude"]
            location_id = int(meta["location_id"])
        else:
            print(f"⚠️ Sensor {sensor_id} not in metadata. Run pipeline 0 (setup) first to generate metadata.")
            failed += 1
            failed_sensors.append((sensor_id, "Missing metadata"))
            continue

        # Fetch weather with retry logic
        weather_df = None
        max_retries = 3
        for attempt in range(max_retries):
            try:
                weather_df = fetchers.get_historical_weather(
                    location_id, start_date, end_date, latitude, longitude
                )
                break  # Success, exit retry loop
            except (ConnectionError, ProtocolError, Timeout) as e:
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 5
                    print(f"⚠️  Network error for sensor {sensor_id}, retrying in {wait_time}s... ({attempt + 1}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to fetch weather for sensor {sensor_id} after {max_retries} attempts: {type(e).__name__}")
                    raise

        if weather_df is None or len(weather_df) == 0:
            print(f"⚠️ No historical weather for sensor {sensor_id}, skipping.")
            failed += 1
            failed_sensors.append((sensor_id, "No weather data"))
            continue

        weather_df["date"] = weather_df["date"].dt.tz_localize(None)

        """ CHECK TYPES """

        weather_df["location_id"] = weather_df["location_id"].astype("int32")
        weather_df = weather_df.astype({
            "location_id": "int32",
            "temperature_2m_mean": "float64",
            "precipitation_sum": "float64",
            "wind_speed_10m_max": "float64",
            "wind_direction_10m_dominant": "float64",
        })

        weather_fg.insert(weather_df)

        # Add location_id to air quality data for joining with weather
        aq_df["location_id"] = location_id


        """ CHECK TYPES """

        aq_df["sensor_id"] = aq_df["sensor_id"].astype("int32")
        aq_df["location_id"] = aq_df["location_id"].astype("int32")
        aq_columns = [f.name for f in air_quality_fg.features]
        aq_df = aq_df[aq_columns]
        aq_df = aq_df.astype({
            "sensor_id": "int32",
            "location_id": "int32",
            "pm25": "float64",
            "pm25_lag_1d": "float64",
            "pm25_lag_2d": "float64",
            "pm25_lag_3d": "float64",
            "pm25_rolling_3d": "float64",
            "pm25_nearby_avg": "float64",
        })

        air_quality_fg.insert(aq_df)

        # Add to processed set
        existing_sensors.add(sensor_id)
        successful += 1

        print(f"✅ Inserted sensor {sensor_id} ({successful}/{remaining} complete, {failed} failed)")

    except (ConnectionError, ProtocolError, Timeout) as e:
        failed += 1
        error_msg = f"Network error: {type(e).__name__}"
        failed_sensors.append((sensor_id, error_msg))
        print(f"❌ {error_msg} for sensor {sensor_id}")
        print(f"   Continuing with next sensor... ({successful} successful, {failed} failed, {skipped} skipped)")
        continue
        
    except RequestException as e:
        failed += 1
        error_msg = f"Request error: {type(e).__name__}"
        failed_sensors.append((sensor_id, error_msg))
        print(f"❌ {error_msg} for sensor {sensor_id}")
        print(f"   Continuing with next sensor... ({successful} successful, {failed} failed, {skipped} skipped)")
        continue
        
    except Exception as e:
        failed += 1
        error_msg = f"{type(e).__name__}: {str(e)[:50]}"
        failed_sensors.append((sensor_id, error_msg))
        print(f"❌ Unexpected error processing sensor {sensor_id}: {type(e).__name__} - {str(e)[:100]}")
        import traceback
        print(f"   Traceback: {traceback.format_exc()[:200]}")
        print(f"   Continuing with next sensor... ({successful} successful, {failed} failed, {skipped} skipped)")
        continue

print(f"\n🎉 Backfill complete!")
print(f"📊 Final Summary:")
print(f"   ✅ Successfully processed: {successful}")
print(f"   ❌ Failed: {failed}")
print(f"   ⏩ Skipped (already processed): {skipped}")
print(f"   📈 Total in feature store: {len(existing_sensors)}/{total_sensors}")

if len(failed_sensors) > 0:
    print(f"\n⚠️  Failed Sensors Detail:")
    for sid, reason in failed_sensors:
        print(f"   • Sensor {sid}: {reason}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (22.56s) 
📋 Found 103 sensors already in feature store
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
📍 Loaded metadata for 103 sensors
📊 Total sensors: 103, Already processed: 103, Remaining: 0
⏩ Skipping sensor 105325, already in feature store
⏩ Skipping sensor 107110, already in feature store
⏩ Skipping sensor 112672, already in feature store
⏩ ... (suppressing further skip messages)

🎉 Backfill complete!
📊 Final Summary:
   ✅ Successfully processed: 0
   ❌ Failed: 0
   ⏩ Skipped (already processed): 103
   📈 Total in feature store: 103/103


## 1.4. Update Descriptions

In [6]:
hopsworks_admin.update_air_quality_description(air_quality_fg)
hopsworks_admin.update_sensor_metadata_description(sensor_metadata_fg)
hopsworks_admin.update_weather_description(weather_fg)

## 1.5. Validation Setup
Creates Great Expectations validation suites for air quality and weather data with column value constraints.

In [ ]:
aq_expectation_suite = gx.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

# pm25 should be >= 0
aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "pm25",
            "min_value": -0.1,
            "max_value": None,
            "strict_min": True,
        },
    )
)

aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "date"},
    )
)

# sensor_id + date should be unique (PK)
aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_compound_columns_to_be_unique",
        kwargs={"column_list": ["sensor_id", "date"]},
    )
)

# rolling + lag features should be numeric (float or int)
for col in ["pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d"]:
    aq_expectation_suite.add_expectation(
        gx.core.ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_in_type_list",
            kwargs={
                "column": col,
                "type_list": ["float", "int"],
                "mostly": 1.0
            },
        )
    )

hopsworks_admin.save_or_replace_expectation_suite(air_quality_fg, aq_expectation_suite)


weather_expectation_suite = gx.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

# Date column - should be dateutil parseable
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "date"},
    )
)

# Temperature column - allow nulls, should be within physical range
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "temperature_2m_mean",
            "min_value": -80,
            "max_value": 60,
            "mostly": 1.0,
            "allow_cross_type_comparisons": True
        },
    )
)

# Precipitation column - should be >= 0, allow nulls
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "precipitation_sum",
            "min_value": 0,
            "max_value": None,
            "mostly": 1.0,          # allow nulls
            "allow_cross_type_comparisons": True
        },
    )
)

# Wind column - should be >= 0, allow nulls
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "wind_speed_10m_max",
            "min_value": 0,
            "max_value": None,
            "mostly": 1.0,          # allow nulls
            "allow_cross_type_comparisons": True
        },
    )
)

hopsworks_admin.save_or_replace_expectation_suite(weather_fg, weather_expectation_suite)

Deleted existing expectation suite for FG 'air_quality'.
Attached expectation suite to Feature Group, edit it at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1911228
Saved expectation suite for FG 'air_quality'.
Deleted existing expectation suite for FG 'weather'.
Attached expectation suite to Feature Group, edit it at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1893863
Saved expectation suite for FG 'weather'.


## 1.6. Create Feature View

In [8]:
def create_feature_view(fs, air_quality_fg, weather_fg):
    query = (
        air_quality_fg.select_all()
        .join(weather_fg.select_all(), on=["location_id", "date"])
    )

    fv = fs.get_or_create_feature_view(
        name="air_quality_complete_fv",
        version=1,
        query=query,
        labels=["pm25"]
    )

    return fv


air_quality_fv = create_feature_view(fs, air_quality_fg, weather_fg)

## 1.7. Load Historical Data

In [9]:
try:
    metadata_df = sensor_metadata_fg.read()
    if len(metadata_df) == 0:
        print("⚠️ No sensor metadata found. Run first-time CSV processing first.")
    else:
        metadata_df = metadata_df.set_index("sensor_id")
        print(f"📍 Loaded metadata for {len(metadata_df)} sensors")
except Exception as e:
    print(f"❌ Error reading sensor metadata: {e}")
    metadata_df = pd.DataFrame()

historical_df = air_quality_fv.get_batch_data()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.33s) 
📍 Loaded metadata for 103 sensors
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.85s) 


## 1.8. Data Exploration

In [11]:
air_quality_df = air_quality_fg.read()
metadata_df = sensor_metadata_fg.read()
weather_df = weather_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.90s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.33s) 


In [12]:
print("🔍 AIR QUALITY DATA EXPLORATION")
print("="*40)

print(f"Shape: {air_quality_df.shape}")
print(f"Date range: {air_quality_df['date'].min().date()} to {air_quality_df['date'].max().date()}")
print(f"Number of unique sensors: {air_quality_df['sensor_id'].nunique()}")
print(f"Countries: {metadata_df['country'].unique()}")
print(f"Cities: {metadata_df['city'].nunique()} unique cities")

print("\n📊 PM2.5 Statistics:")
print(air_quality_df['pm25'].describe())
print(f"Missing values: {air_quality_df['pm25'].isna().sum()}")

print("\n📈 Engineered Features Statistics:")
for col in ['pm25_rolling_3d', 'pm25_lag_1d', 'pm25_lag_2d', 'pm25_lag_3d', 'pm25_nearby_avg']:
    if col in air_quality_df.columns:
        missing = air_quality_df[col].isna().sum()
        print(f"{col}: {missing} missing values ({missing/len(air_quality_df)*100:.1f}%)")


🔍 AIR QUALITY DATA EXPLORATION
Shape: (165910, 9)
Date range: 2019-12-09 to 2026-01-09
Number of unique sensors: 103
Countries: ['Sweden']
Cities: 85 unique cities

📊 PM2.5 Statistics:
count    165910.000000
mean          3.216124
std          11.894608
min           0.000000
25%           0.900000
50%           1.800000
75%           3.500000
max         999.900000
Name: pm25, dtype: float64
Missing values: 0

📈 Engineered Features Statistics:
pm25_rolling_3d: 165910 missing values (100.0%)
pm25_lag_1d: 349 missing values (0.2%)
pm25_lag_2d: 452 missing values (0.3%)
pm25_lag_3d: 555 missing values (0.3%)
pm25_nearby_avg: 165910 missing values (100.0%)


In [13]:
print("🌤️ WEATHER DATA EXPLORATION") 
print("="*40)

print(f"Shape: {weather_df.shape}")
print(f"Date range: {weather_df['date'].min().date()} to {weather_df['date'].max().date()}")
print(f"Number of unique sensors: {metadata_df['sensor_id'].nunique()}")

print("\n🌡️ Weather Statistics:")
for col in ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']:
    if col in weather_df.columns:
        print(f"{col}:")
        print(f"  Range: {weather_df[col].min():.2f} to {weather_df[col].max():.2f}, Mean: {weather_df[col].mean():.2f}, Missing: {weather_df[col].isna().sum()}")

print("\n📍 Geographic Coverage:")
print(f"Latitude range: {metadata_df['latitude'].min():.3f} to {metadata_df['latitude'].max():.3f}, Longitude range: {metadata_df['longitude'].min():.3f} to {metadata_df['longitude'].max():.3f}")

🌤️ WEATHER DATA EXPLORATION
Shape: (118330, 6)
Date range: 2018-06-01 to 2026-01-16
Number of unique sensors: 103

🌡️ Weather Statistics:
temperature_2m_mean:
  Range: -31.06 to 26.12, Mean: 6.99, Missing: 0
precipitation_sum:
  Range: 0.00 to 105.10, Mean: 2.12, Missing: 0
wind_speed_10m_max:
  Range: 0.20 to 63.46, Mean: 18.40, Missing: 0
wind_direction_10m_dominant:
  Range: 0.00 to 360.00, Mean: 184.89, Missing: 0

📍 Geographic Coverage:
Latitude range: 55.477 to 64.944, Longitude range: 11.175 to 20.881


In [14]:
print("🔗 DATA QUALITY & RELATIONSHIPS")
print("="*40)

# Overall data completeness
sensor_day_counts = air_quality_df.groupby('sensor_id')['date'].count()
total_records = len(air_quality_df)
data_completeness = (1 - air_quality_df['pm25'].isna().sum() / total_records) * 100

print(f"📊 Overall Data Quality:")
print(f"Total records: {total_records:,}")
print(f"Data completeness: {data_completeness:.1f}%")
print(f"Days per sensor - Min: {sensor_day_counts.min()}, Median: {sensor_day_counts.median():.0f}, Max: {sensor_day_counts.max()}")
print(f"Sensors with <30 days: {(sensor_day_counts < 30).sum()}, >365 days: {(sensor_day_counts > 365).sum()}")

# Extreme values summary
extreme_count = (air_quality_df['pm25'] > 100).sum()
very_high_count = (air_quality_df['pm25'] > 50).sum()
print(f"\n⚠️ Air Quality Levels:")
print(f"Extreme readings (>100 μg/m³): {extreme_count} ({extreme_count/total_records*100:.1f}%)")
print(f"Very high readings (>50 μg/m³): {very_high_count} ({very_high_count/total_records*100:.1f}%)")

# Seasonal patterns
if len(air_quality_df) > 0:
    # Create temporary month column without modifying original DataFrame
    temp_months = pd.to_datetime(air_quality_df['date']).dt.month
    monthly_pm25 = air_quality_df.groupby(temp_months)['pm25'].mean()
    print(f"\n🗓️ Seasonal Patterns (PM2.5 μg/m³):")
    seasons = {(12,1,2): "Winter", (3,4,5): "Spring", (6,7,8): "Summer", (9,10,11): "Autumn"}
    for months, season in seasons.items():
        season_avg = monthly_pm25[monthly_pm25.index.isin(months)].mean()
        print(f"  {season}: {season_avg:.1f}")

🔗 DATA QUALITY & RELATIONSHIPS
📊 Overall Data Quality:
Total records: 165,910
Data completeness: 100.0%
Days per sensor - Min: 88, Median: 1873, Max: 2187
Sensors with <30 days: 0, >365 days: 100

⚠️ Air Quality Levels:
Extreme readings (>100 μg/m³): 41 (0.0%)
Very high readings (>50 μg/m³): 150 (0.1%)

🗓️ Seasonal Patterns (PM2.5 μg/m³):
  Winter: 3.8
  Spring: 2.7
  Summer: 2.9
  Autumn: 3.5
